**Píldora 1 : Definición de agentes y creación de tareas**

Definición del modelo LLM. Cada tarea se asigna a un agente y define qué debe hacer.

> Creación de los agentes: Investigador de Mercado, Especialista Agronómico, Asesor Financiero Inclusivo, Explicador Inclusivo, Coordinador con Capa RAG

*   MarketResearchOutput
*   AgronomicExpertOutput
*   FinancialAdvisorOutput
*   InclusiveExplainerOutput
*   RAGCoordinatorOutput







In [ ]:
crew = Crew(
    agents=[
        market_researcher,
        agronomic_expert,
        financial_advisor,
        inclusive_explainer,
        rag_coordinator
    ],
    tasks=[
        task_market_research,
        task_agronomic_analysis,
        task_financial_advice,
        task_inclusive_explanations,
        task_rag_support
    ]
)


In [ ]:
result = crew.kickoff(inputs=inputs)


In [ ]:
from models import (
    MarketResearchOutput, AgronomicExpertOutput,
    FinancialAdvisorOutput, InclusiveExplainerOutput,
    RAGCoordinatorOutput
)


In [ ]:
from crewai import Agent, Task, Crew, LLM

llm = Gemini(model="gemini/gemini-2.5-flash", verbose=True)
#    google_api_key="TU API KEY",)

# Agentes para proyecto

market_researcher = Agent(
    role="Investigador del Mercado",
    goal="Analizar precios de cultivos, demanda y tendencias en la Región de Murcia",
    backstory="Eres un experto en análisis económico y de mercado agrícola para la cuenca hidrográfica del Segura.",
    llm=llm,
    allow_delegation=False,
    verbose=True
)

agronomic_expert = Agent(
    role="Especialista Agronomo",
    goal="Proporcionar datos técnicos sobre cultivos, clima y suelo adaptados a agricultores locales",
    backstory="Tienes experiencia en agricultura mediterránea, especialmente en la zona de Murcia y del levante español, proporcionas asesoramiento a pequeños agricultores.",
    llm=llm,
    allow_delegation=False,
    verbose=True
)

financial_advisor = Agent(
    role="Asesor Financiero Inclusivo",
    goal="Recomendar opciones financieras accesibles y claras para todo tipo de agricultores, evitando la exclusión financiera",
    backstory="Eres un especialista en finanzas agrícolas y del campo con foco en igualdad de acceso y lenguaje entendible para usuarios diversos.",
    llm=llm,
    allow_delegation=False,
    verbose=True
)

inclusive_explainer = Agent(
    role="Agente Explicativo Inclusivo",
    goal="Transformar las recomendaciones técnicas en explicaciones claras y accesibles para todos los usuarios",
    backstory="Eres experto en comunicación clara y simplificada, facilitando la comprensión a usuarios con diversos niveles de educación y acceso.",
    llm=llm,
    allow_delegation=False,
    verbose=True
)

rag_coordinator = Agent(
    role="Coordinador con Capa RAG",
    goal="Gestionar la recuperación de documentos y datos actualizados para soportar las recomendaciones con evidencia local",
    backstory="Manejas bases de datos y documentos normativos y técnicos para enriquecer las respuestas con soporte documental fiable.",
    llm=llm,
    allow_delegation=False,
    verbose=True
)

# Ejemplo breve de Crew que orquesta los agentes (concepto)

crew = Crew(agents=[
    market_researcher,
    agronomic_expert,
    financial_advisor,
    inclusive_explainer,
    rag_coordinator
])

# Aquí el resto del código implementaría la interaccion entre agentes,
# la recepción de perfil del agricultor, y la generación final de la recomendación
# incluyendo la capa RAG para respaldo documental.

# Ejemplo de creación de tarea:
# task = Task("Recomendar opción financiera para agricultor de uva en Murcia", ...)
# result = crew.execute(task)
# print(result)


**Paso 3: Definición de Modelos Pydantic para las salidas de cada agente**

Los agentes trabajan en secuencia: Buscador → Asesor → Cerrador.

Definimos las salidas de cada agente:

*   MarketResearchOutput
*   AgronomicExpertOutput
*   FinancialAdvisorOutput
*   InclusiveExplainerOutput
*   RAGCoordinatorOutput







In [ ]:
# Importar Pydantic
from pydantic import BaseModel
from typing import List, Dict

# La clase Pydantic de salida de agente de  Investigación de Mercado (esto va en el archivo models.py)
class MarketResearchOutput(BaseModel):
    problems: List[str]           # Lista de problemas clave identificados en el mercado agrícola de Murcia
    data: Dict[str, float]        # Datos cuantitativos que respaldan cada problema (por ejemplo, precios, porcentajes)

# Salida del Agente Especialista Agronómico
class AgronomicExpertOutput(BaseModel):
    crop_recommendations: List[str]      # Recomendaciones para tipos de cultivos según condiciones locales
    climate_data: Dict[str, float]       # Datos climáticos relevantes (temperaturas medias, lluvias, etc.)
    soil_conditions: Dict[str, str]      # Características del suelo (tipo, pH, etc.)

# Salida del Agente Asesor Financiero Inclusivo
class FinancialAdvisorOutput(BaseModel):
    financial_options: List[str]          # Opciones financieras disponibles y accesibles para agricultores
    cost_estimations: Dict[str, float]    # Estimación de costos y tasas para cada opción

# Salida del Agente Explicativo Inclusivo
class InclusiveExplainerOutput(BaseModel):
    simplified_explanations: List[str]   # Explicaciones claras y accesibles para los usuarios sobre las recomendaciones financieras y agrícolas

# Salida del Agente Coordinador con Capa RAG
class RAGCoordinatorOutput(BaseModel):
    document_references: List[str]       # Lista de documentos consultados o referencias usadas para las recomendaciones
    context_summaries: List[str]         # Resúmenes del contenido recuperado para soporte documental

# Ejemplo de uso en funciones o métodos:
# market_output = MarketResearchOutput(problems=["Alta variabilidad de precios"], data={"uva": 1.2, "citrico": 0.8})
# agronomic_output = AgronomicExpertOutput(crop_recommendations=["uva", "olivo"], climate_data={"temp_media": 20.5}, soil_conditions={"tipo": "arcilloso"})
# financial_output = FinancialAdvisorOutput(financial_options=["microcrédito", "subsidio regional"], cost_estimations={"microcrédito": 0.07, "subsidio regional": 0.02})
# inclusive_output = InclusiveExplainerOutput(simplified_explanations=["El microcrédito tiene un interés moderado y es fácil de acceder."])
# rag_output = RAGCoordinatorOutput(document_references=["Normativa local 2024"], context_summaries=["El subsidio regional financia cultivos sostenibles."])


**Paso 4. Creación del Crew multiagente**`

Agrupación de agentes y tareas

In [ ]:
from crewai import Agent, Task, Crew
#from crewai_tools import DirectoryReadTool, FileReadTool, WebsiteSearchTool

# Importar modelos Pydantic definidos para los outputs
from models import (
    MarketResearchOutput, AgronomicExpertOutput,
    FinancialAdvisorOutput, InclusiveExplainerOutput,
    RAGCoordinatorOutput
)

# Suponiendo que los agentes están definidos como en código previo:
# market_researcher, agronomic_expert, financial_advisor, inclusive_explainer, rag_coordinator

# Definición de las tareas para cada agente

task_market_research = Task(
    description="Analiza problemas y condiciones del mercado agrícola en Murcia.",
    expected_output="Lista de problemas clave con datos cuantificables que afectan a los agricultores.",
    pydantic_output_model=MarketResearchOutput,
    agent=market_researcher,
    tools=[]  # Puedes añadir herramientas si deseas
)

task_agronomic_analysis = Task(
    description="Proporciona recomendaciones técnicas agronómicas para cultivos y clima de Murcia.",
    expected_output="Recomendaciones de cultivos, clima y condición del suelo localizadas.",
    pydantic_output_model=AgronomicExpertOutput,
    agent=agronomic_expert,
    tools=[]
)

task_financial_advice = Task(
    description="Recomienda opciones financieras claras y accesibles para agricultores locales.",
    expected_output="Opciones financieras disponibles detalladas con costos estimados.",
    pydantic_output_model=FinancialAdvisorOutput,
    agent=financial_advisor,
    tools=[]
)

task_inclusive_explanations = Task(
    description="Convierte las recomendaciones técnicas y financieras en explicaciones sencillas para todos los usuarios.",
    expected_output="Explicaciones claras y accesibles para usuarios de diversos niveles educativos y tecnológicos.",
    pydantic_output_model=InclusiveExplainerOutput,
    agent=inclusive_explainer,
    tools=[]
)

task_rag_support = Task(
    description="Gestiona la recuperación y resumen de documentos normativos y técnicos que soportan las recomendaciones.",
    expected_output="Referencias y resúmenes de documentos para validar y respaldar las recomendaciones.",
    pydantic_output_model=RAGCoordinatorOutput,
    agent=rag_coordinator,
    tools=[]
)

# Crear la tripulación con agentes y tareas asignadas
crew = Crew(
    agents=[
        market_researcher,
        agronomic_expert,
        financial_advisor,
        inclusive_explainer,
        rag_coordinator
    ],
    tasks=[
        task_market_research,
        task_agronomic_analysis,
        task_financial_advice,
        task_inclusive_explanations,
        task_rag_support
    ]
)

# Ahora puedes crear una tarea general para que la tripulación la ejecute coordinadamente.
# Por ejemplo:
# main_task = Task(description="Recomendar soluciones integrales para agricultores en Murcia.", ...)
# results = crew.execute(main_task)
# print(results)


In [ ]:
crew = Crew(
    agents=[
        market_researcher,
        agronomic_expert,
        financial_advisor,
        inclusive_explainer,
        rag_coordinator
    ],
    tasks=[
        task_market_research,
        task_agronomic_analysis,
        task_financial_advice,
        task_inclusive_explanations,
        task_rag_support
    ]
)


In [ ]:
result = crew.kickoff(inputs=inputs)


**Paso 5. Inputs de ejemplo para el usuario agricultor**

Ejecución de la tripulación y obtención de resultados

In [ ]:
# Definición de inputs para la tripulación de agentes
inputs = {
    'sector': "agricultura",
    'region': "Murcia",
    'profile': {
        "tipo_cultivo": "uva",
        "tamaño_explotacion": "pequeña",
        "nivel_educativo": "bajo",
        "necesidad_financiera": True
    }
}

# Lanzamiento de la ejecución coordinada de los agentes
result = crew.kickoff(inputs=inputs)

# Mostrar resultado general
print(result)


Nota: Este ejemplo pasa un diccionario de inputs que puede ser usado por los agentes para contextualizar sus respuestas, como sector agrícola, región geográfica concreta, y perfil del agricultor con datos útiles para generar recomendaciones inclusivas y con soporte RAG.

Se puede ampliar el diccionario de inputs con más parámetros según se necesite para enriquecer las tareas de los agentes.